# SageMaker で提供される DeepAR アルゴリズムの学習と推論を行う

#### ノートブックに含まれる内容

- Amazon が提供するアルゴリズムの使いかた
- 中でも，DeepAR アルゴリズムの使い方

#### ノートブックで使われている手法の詳細

- アルゴリズム: DeepAR
- データ: ランダム時系列を作成
- 可視化手段: matplotlib


## Deep AR とは

DeepAR とは，AWS から提供されている state-of-the-art の時系列データ予測アルゴリズムです．このノートブックでは，DeepAR を使って，モデルの学習と推論を行います．DeepAR は，スカラの時系列データに対する予測を行う，教師あり学習アルゴリズムです．古典的な ARIMA モデルや指数平滑法は，単一時系列のみを学習に用いており，そのためしばしば極端な予測値を出力します．DeepAR は複数の時系列を入力として取り，併せて学習することで，より安定した予測結果を出すことができます．そのため，例えば多種の製品の需要，サーバ負荷，Webページへのリクエスト数といった複数の時系列が手に入るような場合に，非常に有効です．

DeepAR の詳細については，[公式ドキュメントの解説](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html)および [arXiv の論文](https://arxiv.org/abs/1704.04110)をご覧ください

## セットアップ

In [ ]:
import time
import numpy as np
np.random.seed(1)
import pandas as pd
import json
import matplotlib.pyplot as plt

ここでは SageMaker SDK を使って作業を行います．また，学習データを S3 にアップロードする際に，s3fs モジュールを使用します．当該モジュールがインスタンスに入っていないため，ここでは conda コマンドでインストールします（もちろん，その他のモジュールも `pip` コマンドでインストールすることができます）．

In [ ]:
!conda install -y s3fs

In [ ]:
import boto3
import s3fs
import sagemaker
from sagemaker import get_execution_role

データを置くバケットを指定します．このバケットは，ノートブックインスタンスがある（そして，学習と推論を行う）リージョンと同一である必要があります．また SageMaker セッションで使用する IAM role に，データに対するアクセス権限を与える必要があります．ここでは `get_execution_role` を使って，ノートブックインスタンス作成時に割り当てたロールを使用します．

以下の**<span style="color: red;"> 1 行目のバケット名について，`sagemaker-ap-northeast-1-handson-YYYYMMDD-XX` の `YYYYMMDD` を今日の日付に，`XX`にはIAMユーザ名などをいれて一意な名前となるようにしてください</span>**。

In [ ]:
bucket = 'sagemaker-ap-northeast-1-handson-YYYYMMDD-XX'
prefix = 'sagemaker/DEMO-deepar'

sagemaker_session = sagemaker.Session()
role = get_execution_role()

boto3.resource('s3').create_bucket(Bucket=bucket, CreateBucketConfiguration={'LocationConstraint': 'ap-northeast-1'})
s3_data_path = "{}/{}/data".format(bucket, prefix)
s3_output_path = "{}/{}/output".format(bucket, prefix)

次に，当該リージョンで提供されているコンテナイメージを取得します．AWS から提供されているイメージについては，[公式ドキュメントに一覧](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html)があります．

In [ ]:
containers = {
    'us-east-1': '522234722520.dkr.ecr.us-east-1.amazonaws.com/forecasting-deepar:latest',
    'us-east-2': '566113047672.dkr.ecr.us-east-2.amazonaws.com/forecasting-deepar:latest',
    'us-west-2': '156387875391.dkr.ecr.us-west-2.amazonaws.com/forecasting-deepar:latest',
    'eu-west-1': '224300973850.dkr.ecr.eu-west-1.amazonaws.com/forecasting-deepar:latest',
    'ap-northeast-1': '633353088612.dkr.ecr.ap-northeast-1.amazonaws.com/forecasting-deepar:latest',
}
image_name = containers[boto3.Session().region_name]

## データセットのダウンロード
データセットをダウンロードしてnotebookインスタンスと同じ場所に置きます。手元のクライアントPCにダウンロードしてからJupyterにアップロードすることも可能ですが、直接、notebookインスタンスの場所にダウンロードすることができます。何かをダウンロードしたいときは  
```
!wget (ダウンロードしたいファイルのURL)
```
とします。`wget`はファイルをダウンロードするコマンドで、冒頭に`!`をつけるとJupyter上で実行することができます。ここでは`data`というフォルダを作って、そこにダウンロードしましょう。以下のセルを選択して実行（ctrl+Enter、shift+Enterまたは、上のメニューからRunをクリック）してください。

In [ ]:
!mkdir data
!cd data && wget https://archive.ics.uci.edu/ml/machine-learning-databases/00381/PRSA_data_2010.1.1-2014.12.31.csv

## データセットをのぞいてみる
データセットにどういうデータが含まれていて、それらがどういう風に表現されているのかを確認してみましょう。データの可視化や処理には`pandas`を使うのが便利です。例えば以下のことに気がつくと思います。
- pm2.5には観測値がない (NaN）ところがあるので、これらをそのまま使えない。
- 風向きはカテゴリ変数なのでダミー変数に置き換える必要がある。

In [ ]:
import pandas as pd
from IPython.display import display
data = pd.read_csv('./data/PRSA_data_2010.1.1-2014.12.31.csv')
display(data.head())

## 前処理を実行する

ここでは以下のような前処理を実行します。
- 年〜時までの時間に関するデータをマージしてindexにする。
- カテゴリ変数をone-hot encodingする
- 不要になった列を消す（Noや時間に関する列など）
- NaNを含む行を削除し、線形補間する。

最終的なレコード数と特徴数を表示します。

In [ ]:
from datetime import datetime
data['time'] = data.apply(lambda x: datetime(
                          x['year'], x['month'], x['day'], x['hour']), axis=1)
data.set_index('time', inplace = True) 

# Categoriacal variable is converted into dummy variables.
cbwd_dummy = pd.get_dummies(data['cbwd'])
data = pd.concat([data, cbwd_dummy], axis = 1)

# Unnecessary variables are dropped here.
data.drop(columns=['No', 'cbwd', 'year', 'month', 'day', 'hour'], inplace =True)

# Rows including n/a are dropped.
data.dropna(inplace=True)

# Linear interploation
data = data.resample('h').interpolate(method='linear')

print("Rows: {}, Columns: {}".format(len(data.index), len(data.columns)))
display(data.head())

## 学習/テストデータの分割

学習/テストデータの分割については2013年までを学習データ、2014年をトレーニングデータとします。

In [ ]:
train_data = data[data.index.year <2014]
test_data = data[data.index.year ==2014]

学習データの最初の100データのpm2.5だけ表示してみます。（pm2.5のindexは0です）

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_data.index[:100], train_data.values[:100, 0], label="pm2.5")
plt.legend()
plt.show()

`pandas.Series` フォーマットの時系列データを[Deep AR で指定している JSON 形式](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html)に変換して，S3 にアップロードします。

In [ ]:
def series_to_obj(ts, cat):
    obj = {"start": str(ts.index[0]), "target": list(ts)}
    obj["cat"] = cat
    return obj

def series_to_jsonline(ts, cat=None):
    return json.dumps(series_to_obj(ts, cat))

In [ ]:
encoding = "utf-8"
s3filesystem = s3fs.S3FileSystem()

with s3filesystem.open(s3_data_path + "/train/train.json", 'wb') as fp:
        for i in range(len(train_data.columns)):
            ts = train_data.iloc[:, i]
            fp.write(series_to_jsonline(ts, i).encode(encoding))
            fp.write('\n'.encode(encoding))


with s3filesystem.open(s3_data_path + "/test/test.json", 'wb') as fp:
        for i in range(len(test_data.columns)):
            ts = test_data.iloc[:, i]
            fp.write(series_to_jsonline(ts, i).encode(encoding))
            fp.write('\n'.encode(encoding))

## モデルの学習

まず，`Estimator` オブジェクトを作成します

In [ ]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.xlarge',
    base_job_name='DEMO-deepar',
    output_path="s3://" + s3_output_path
)

作成した `Estimator` に対して，ハイパーパラメタをセットします．

In [ ]:
hyperparameters = {
    "time_freq": 'H',
    "context_length": 72,
    "prediction_length": 12,
    "num_cells": "40",
    "num_layers": "3",
    "likelihood": "gaussian",
    "epochs": "20",
    "mini_batch_size": "32",
    "learning_rate": "0.001",
    "dropout_rate": "0.05",
    "early_stopping_patience": "10",
    "embedding_dimension": "10",
    "cardinality": len(train_data.columns)
}

In [ ]:
estimator.set_hyperparameters(**hyperparameters)

学習に使用するデータを指定して，ジョブを開始します．この際に `test` データチャネルを指定することで，学習済みモデルをテストデータに適用した際の，accuracy を出力してくれます．

In [ ]:
data_channels = {
    "train": "s3://{}/train/".format(s3_data_path),
    "test": "s3://{}/test/".format(s3_data_path)
}

estimator.fit(inputs=data_channels)

## モデルの推論を実行

In [ ]:
job_name = estimator.lat est_training_job.name

endpoint_name = sagemaker_session.endpoint_from_job(
    job_name=job_name,
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    deployment_image=image_name,
    role=role
)

推論の際に，`pandas.Series` オブジェクトを入力として渡せるようにするための，ラッパークラスを定義します．本来は DeepAR で指定された JSON フォーマットなためです．

In [ ]:
class DeepARPredictor(sagemaker.predictor.RealTimePredictor):

    def set_prediction_parameters(self, freq, prediction_length):
        """Set the time frequency and prediction length parameters. This method **must** be called
        before being able to use `predict`.
        
        Parameters:
        freq -- string indicating the time frequency
        prediction_length -- integer, number of predicted time points
        
        Return value: none.
        """
        self.freq = freq
        self.prediction_length = prediction_length
        
    def predict(self, ts, cat=None, encoding="utf-8", num_samples=100, quantiles=["0.1", "0.5", "0.9"]):
        """Requests the prediction of for the time series listed in `ts`, each with the (optional)
        corresponding category listed in `cat`.
        
        Parameters:
        ts -- list of `pandas.Series` objects, the time series to predict
        cat -- list of integers (default: None)
        encoding -- string, encoding to use for the request (default: "utf-8")
        num_samples -- integer, number of samples to compute at prediction time (default: 100)
        quantiles -- list of strings specifying the quantiles to compute (default: ["0.1", "0.5", "0.9"])
        
        Return value: list of `pandas.DataFrame` objects, each containing the predictions
        """
        prediction_times = [ts.iloc[:, i].index[-1]+1 for  i in range(len(ts.columns))]
        req = self.__encode_request(ts, cat, encoding, num_samples, quantiles)
        res = super(DeepARPredictor, self).predict(req)
        return self.__decode_response(res, prediction_times, encoding)
    
    def __encode_request(self, ts, cat, encoding, num_samples, quantiles):
        instances = [series_to_obj(ts.iloc[:,k], k) for k in range(len(ts.columns))]
        configuration = {"num_samples": num_samples, "output_types": ["quantiles"], "quantiles": quantiles}
        http_request_data = {"instances": instances, "configuration": configuration}
        return json.dumps(http_request_data).encode(encoding)
    
    def __decode_response(self, response, prediction_times, encoding):
        response_data = json.loads(response.decode(encoding))
        list_of_df = []
        for k in range(len(prediction_times)):
            prediction_index = pd.DatetimeIndex(start=prediction_times[k], freq=self.freq, periods=self.prediction_length)
            list_of_df.append(pd.DataFrame(data=response_data['predictions'][k]['quantiles'], index=prediction_index))
        return list_of_df

In [ ]:
predictor = DeepARPredictor(
    endpoint=endpoint_name,
    sagemaker_session=sagemaker_session,
    content_type="application/json"
)
freq = 'H'
predictor.set_prediction_parameters(freq, prediction_length=12)

実際に予測を行って結果をグラフに表示します．今回は2010-2013年の学習データを入れて、2014年の最初の12時間の予測を行います。予測対象はpm2.5の値のみです。同時刻の実データをtest_dataから抽出して比較します。

In [ ]:
# 訓練に使った２０１０年から2013年末のデータを入れる
# テストに含まれる2014年の実測値のみを比較する。
#print(train_data)
list_of_df = predictor.predict(train_data)
#print(list_of_df)
predict = list_of_df[0]
predict2014 = predict[predict.index.year == 2014]
actual2014 = test_data.loc[predict2014.index]

In [ ]:
plt.figure(figsize=(12,6))
p10 = predict2014['0.1']
p90 = predict2014['0.9']
plt.fill_between(p10.index, p10, p90, color='y', alpha=0.5, label='80% confidence interval')
plt.plot(p10.index, predict2014['0.5'], label='prediction median')
plt.plot(p10.index, actual2014['pm2.5'], label='ground truth')
plt.legend()
plt.show()

次に2014年のテストデータを入れて1時間先のデータを逐次予測していきます。合計24時間のデータを予測します。

In [ ]:
start_time=0
end_time = 24
one_step_predict = pd.DataFrame()
one_step_time = []
for i in range(start_time, end_time):
    print("\r prediction: {}/{} done".format(i+1 - start_time, end_time-start_time), end="")
    list_of_df = predictor.predict(test_data.iloc[i: i+72])
    one_step_predict = pd.concat([one_step_predict, list_of_df[0].iloc[0:1]])

one_step_predict2014 = one_step_predict
one_step_actual2014 = test_data.loc[one_step_predict.index]

plt.figure(figsize=(12,6))
p10 = one_step_predict2014['0.1']
p90 = one_step_predict2014['0.9']
plt.fill_between(p10.index, p10, p90, color='y', alpha=0.5, label='80% confidence interval')
plt.plot(p10.index, one_step_predict2014['0.5'], label='prediction median')
plt.plot(p10.index, one_step_actual2014['pm2.5'], label='ground truth')
plt.legend()
plt.show()

## エンドポイントの削除

全て終わったら，エンドポイントを削除します．

In [ ]:
sagemaker_session.delete_endpoint(endpoint_name)